In [24]:
import pandas as pd
import random
import sklearn

from pathlib import Path
import xgboost as xgb
from xgboost.callback import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC


In [25]:
CATEGORICAL_FEATURE_NAMES = [
    "gender",
    "hypertension",
    "heart_disease",
    "ever_married",
    "work_type",
    "residence_type",
    "smoking_status",
]

NUMERIC_FEATURE_NAMES = ["age", "avg_glucose_level", "bmi"]


def encode_int(data: pd.DataFrame, categorical_features: list[str]):
    return pd.get_dummies(data, columns=categorical_features, drop_first=True)


def split_label(data: pd.DataFrame):
    x = data.copy().drop("stroke", axis=1)
    y = data["stroke"]  # labels

    return x, y


def resample(data: pd.DataFrame, seed: int, categorical_features: list[str]):
    """oversample positive cases with SMOTE and undersample negative with EEN"""
    # encode categorical features first
    enc = OrdinalEncoder()
    data[categorical_features] = enc.fit_transform(data[categorical_features])

    X = data.drop(columns=["stroke"], axis=1)
    Y = data["stroke"]

    cat_features_indices = [
        data.columns.get_loc(label) for label in categorical_features
    ]

    smote_nc = SMOTENC(categorical_features=cat_features_indices, random_state=seed)
    smote_een = SMOTEENN(smote=smote_nc, random_state=seed, sampling_strategy="auto")

    x_resampled, y_resampled = smote_een.fit_resample(X, Y)
    x_resampled["stroke"] = y_resampled

    x_resampled[categorical_features] = enc.inverse_transform(
        x_resampled[categorical_features]
    )

    return pd.DataFrame(x_resampled, columns=data.columns)


def scale(df):
    X_num = df[NUMERIC_FEATURE_NAMES]
    X_cat = df[CATEGORICAL_FEATURE_NAMES]

    scaler = StandardScaler()
    scaler.fit(X_num)

    X_scaled = scaler.transform(X_num)
    X_scaled = pd.DataFrame(X_scaled, index=X_num.index, columns=X_num.columns)

    df_scaled = pd.concat([X_scaled, X_cat, df["stroke"]], axis=1)

    return df_scaled


def split_train_valid_test(data_df, seed: int, resample_training: bool):
    data_df = data_df.sample(frac=1, random_state=seed)

    test_set = data_df[round(len(data_df) * 0.85) :]
    train_validation_data = data_df[: round(len(data_df) * 0.85)].sample(
        frac=1, random_state=seed
    )

    train_set = train_validation_data[: round(len(data_df) * 0.70)]
    validation_set = train_validation_data[round(len(data_df) * 0.70) :]

    if resample_training:
        train_set = resample(
            train_set, seed, categorical_features=CATEGORICAL_FEATURE_NAMES
        )

    return train_set, validation_set, test_set

def split_train_valid_test_stratified(data_df, seed: int, resample_training: bool):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=seed)
    d_x, d_y = split_label(data_df)

    train_index, valid_test_index = list(sss.split(d_x, d_y))[0]

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=seed)

    v_t_x, v_t_y = split_label(data_df.iloc[valid_test_index])

    validation_index, test_index = list(sss.split(v_t_x, v_t_y))[0]

    train_df = data_df.iloc[train_index.tolist()]
    validation_df = data_df.iloc[validation_index.tolist()]
    test_df = data_df.iloc[test_index.tolist()]

    if resample_training:
        train_set = resample(
            train_df, seed, categorical_features=CATEGORICAL_FEATURE_NAMES
        )

    return train_df, validation_df, test_df


def one_hot_encode(
    data_df: pd.DataFrame,
    train_df: pd.DataFrame,
    validation_df: pd.DataFrame,
    test_df: pd.DataFrame,
):
    # ensure train, validation, and test all have the same columns after ohe

    ohe_columns = encode_int(data_df, CATEGORICAL_FEATURE_NAMES).columns

    train_df_e, validation_df_e, test_df_e = [
        pd.DataFrame(
            encode_int(data=df, categorical_features=CATEGORICAL_FEATURE_NAMES),
            columns=ohe_columns,
        ).fillna(0)
        for df in [train_df, validation_df, test_df]
    ]

    return train_df_e, validation_df_e, test_df_e


def prepare_data(seed: int, resample_training: bool):
    data_df = pd.read_csv(Path().resolve().joinpath("dataset/full_data_clean.csv"))

    train_df, validation_df, test_df = split_train_valid_test_stratified(
        data_df, seed, resample_training
    )

    train_df, validation_df, test_df = [
        scale(df) for df in [train_df, validation_df, test_df]
    ]

    return one_hot_encode(data_df, train_df, validation_df, test_df)


In [26]:
RESULT_COLS = [
    "precision",
    "recall",
    "fscore",
    "accuracy",
    "auc",
    "miss_rate",
    "fall_out_rate",
]
NUM_EXPERIMENTS = 10
NUM_ROUNDS = 20


def metrics(y_pred, p_pred, y_true):
    tn, fp, fn, tp = confusion_matrix(y_pred=y_pred, y_true=y_true).ravel()

    # metrics
    auc = roc_auc_score(y_true=y_true, y_score=p_pred)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    fscore = 2 * tp / (2 * tp + fp + fn)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    miss_rate = fn / (tn + tp)
    fall_out_rate = fp / (fp + tn)

    # return
    return [precision, recall, fscore, accuracy, auc, miss_rate, fall_out_rate]


In [27]:
def run_experiment(seed, experiment, train_data, validation_data, test_data):
    # split labels
    train_validation = pd.concat([train_data, validation_data]).sample(
        frac=1, random_state=seed
    )

    x, y = split_label(train_validation)
    x_test, y_test = split_label(test_data)

    # metrics
    param = {"max_depth": 2, "eta": 1, "objective": "binary:logistic"}

    es = EarlyStopping(
        rounds=3,
        save_best=True,
        maximize=True,
        data_name="validation_0",
        metric_name="auc",
    )

    clf = xgb.sklearn.XGBClassifier(
        learning_rate=0.1,
        objective=param["objective"],
        gpu_id=0,
        predictor="gpu_predictor",
    )

    clf.fit(x, y, eval_set=[(x_test, y_test), (x, y)])

    y_pred = clf.predict(x_test)
    p_pred = clf.predict_proba(x_test)[:, 1]

    return y_pred, p_pred, y_test


In [28]:
# evaluate xgboost

results = {}
results_resample = {}

for experiment in range(NUM_EXPERIMENTS):
    seed = random.randint(0, 1000)

    train_data, validation_data, test_data = prepare_data(
        seed=experiment, resample_training=False
    )

    y_pred, p_pred, y_test = run_experiment(
        seed, experiment, train_data, validation_data, test_data
    )

    results[experiment] = list(metrics(y_pred, p_pred, y_test))

    train_data, validation_data, test_data = prepare_data(
        seed=experiment, resample_training=True
    )

    y_pred, p_pred, y_test = run_experiment(
        seed, experiment, train_data, validation_data, test_data
    )

    results_resample[experiment] = list(metrics(y_pred, p_pred, y_test))



[0]	validation_0-logloss:0.60456	validation_1-logloss:0.60441
[1]	validation_0-logloss:0.53207	validation_1-logloss:0.53173
[2]	validation_0-logloss:0.47171	validation_1-logloss:0.47119
[3]	validation_0-logloss:0.42083	validation_1-logloss:0.42013
[4]	validation_0-logloss:0.37750	validation_1-logloss:0.37662
[5]	validation_0-logloss:0.34031	validation_1-logloss:0.33926
[6]	validation_0-logloss:0.30823	validation_1-logloss:0.30695
[7]	validation_0-logloss:0.28032	validation_1-logloss:0.27888
[8]	validation_0-logloss:0.25599	validation_1-logloss:0.25435
[9]	validation_0-logloss:0.23467	validation_1-logloss:0.23292
[10]	validation_0-logloss:0.21599	validation_1-logloss:0.21406
[11]	validation_0-logloss:0.19955	validation_1-logloss:0.19748
[12]	validation_0-logloss:0.18506	validation_1-logloss:0.18281
[13]	validation_0-logloss:0.17232	validation_1-logloss:0.16980
[14]	validation_0-logloss:0.16107	validation_1-logloss:0.15830
[15]	validation_0-logloss:0.15109	validation_1-logloss:0.14806
[1

/tmp/ipykernel_12706/3160611095.py:19: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
/tmp/ipykernel_12706/2430262989.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[categorical_features] = enc.fit_transform(data[categorical_features])


[0]	validation_0-logloss:0.60472	validation_1-logloss:0.60442
[1]	validation_0-logloss:0.53234	validation_1-logloss:0.53177
[2]	validation_0-logloss:0.47209	validation_1-logloss:0.47128
[3]	validation_0-logloss:0.42128	validation_1-logloss:0.42024
[4]	validation_0-logloss:0.37803	validation_1-logloss:0.37677
[5]	validation_0-logloss:0.34090	validation_1-logloss:0.33942
[6]	validation_0-logloss:0.30881	validation_1-logloss:0.30718
[7]	validation_0-logloss:0.28102	validation_1-logloss:0.27914
[8]	validation_0-logloss:0.25669	validation_1-logloss:0.25470
[9]	validation_0-logloss:0.23549	validation_1-logloss:0.23329
[10]	validation_0-logloss:0.21692	validation_1-logloss:0.21449
[11]	validation_0-logloss:0.20062	validation_1-logloss:0.19795
[12]	validation_0-logloss:0.18611	validation_1-logloss:0.18336
[13]	validation_0-logloss:0.17330	validation_1-logloss:0.17044
[14]	validation_0-logloss:0.16199	validation_1-logloss:0.15902
[15]	validation_0-logloss:0.15199	validation_1-logloss:0.14890
[1

/tmp/ipykernel_12706/3160611095.py:19: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0]	validation_0-logloss:0.60415	validation_1-logloss:0.60467
[1]	validation_0-logloss:0.53122	validation_1-logloss:0.53222
[2]	validation_0-logloss:0.47044	validation_1-logloss:0.47188
[3]	validation_0-logloss:0.41919	validation_1-logloss:0.42097
[4]	validation_0-logloss:0.37549	validation_1-logloss:0.37762
[5]	validation_0-logloss:0.33796	validation_1-logloss:0.34037
[6]	validation_0-logloss:0.30550	validation_1-logloss:0.30816
[7]	validation_0-logloss:0.27727	validation_1-logloss:0.28019
[8]	validation_0-logloss:0.25261	validation_1-logloss:0.25579
[9]	validation_0-logloss:0.23102	validation_1-logloss:0.23441
[10]	validation_0-logloss:0.21205	validation_1-logloss:0.21562
[11]	validation_0-logloss:0.19534	validation_1-logloss:0.19907
[12]	validation_0-logloss:0.18057	validation_1-logloss:0.18445
[13]	validation_0-logloss:0.16751	validation_1-logloss:0.17149
[14]	validation_0-logloss:0.15598	validation_1-logloss:0.16005
[15]	validation_0-logloss:0.14578	validation_1-logloss:0.14986
[1

...

/tmp/ipykernel_12706/3160611095.py:19: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0]	validation_0-logloss:0.60446	validation_1-logloss:0.60441
[1]	validation_0-logloss:0.53195	validation_1-logloss:0.53176
[2]	validation_0-logloss:0.47154	validation_1-logloss:0.47127
[3]	validation_0-logloss:0.42061	validation_1-logloss:0.42026
[4]	validation_0-logloss:0.37722	validation_1-logloss:0.37676
[5]	validation_0-logloss:0.34000	validation_1-logloss:0.33940
[6]	validation_0-logloss:0.30776	validation_1-logloss:0.30712
[7]	validation_0-logloss:0.27983	validation_1-logloss:0.27906
[8]	validation_0-logloss:0.25545	validation_1-logloss:0.25457
[9]	validation_0-logloss:0.23408	validation_1-logloss:0.23311
[10]	validation_0-logloss:0.21532	validation_1-logloss:0.21427
[11]	validation_0-logloss:0.19885	validation_1-logloss:0.19768
[12]	validation_0-logloss:0.18430	validation_1-logloss:0.18298
[13]	validation_0-logloss:0.17142	validation_1-logloss:0.16998
[14]	validation_0-logloss:0.16002	validation_1-logloss:0.15844
[15]	validation_0-logloss:0.14991	validation_1-logloss:0.14823
[1

/tmp/ipykernel_12706/3160611095.py:19: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
/tmp/ipykernel_12706/2430262989.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[categorical_features] = enc.fit_transform(data[categorical_features])


[0]	validation_0-logloss:0.60457	validation_1-logloss:0.60449
[1]	validation_0-logloss:0.53208	validation_1-logloss:0.53190
[2]	validation_0-logloss:0.47205	validation_1-logloss:0.47144
[3]	validation_0-logloss:0.42112	validation_1-logloss:0.42045
[4]	validation_0-logloss:0.37776	validation_1-logloss:0.37701
[5]	validation_0-logloss:0.34055	validation_1-logloss:0.33972
[6]	validation_0-logloss:0.30861	validation_1-logloss:0.30748
[7]	validation_0-logloss:0.28069	validation_1-logloss:0.27949
[8]	validation_0-logloss:0.25632	validation_1-logloss:0.25505
[9]	validation_0-logloss:0.23501	validation_1-logloss:0.23365
[10]	validation_0-logloss:0.21633	validation_1-logloss:0.21487
[11]	validation_0-logloss:0.19996	validation_1-logloss:0.19834
[12]	validation_0-logloss:0.18550	validation_1-logloss:0.18373
[13]	validation_0-logloss:0.17272	validation_1-logloss:0.17081
[14]	validation_0-logloss:0.16141	validation_1-logloss:0.15933
[15]	validation_0-logloss:0.15141	validation_1-logloss:0.14908
[1

/tmp/ipykernel_12706/3160611095.py:19: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


In [29]:
results = (
    pd.DataFrame()
    .from_dict(results, orient="index", columns=RESULT_COLS)
    .assign(classifier="XGBoost")
)
results_resample = (
    pd.DataFrame()
    .from_dict(results_resample, orient="index", columns=RESULT_COLS)
    .assign(classifier="XGBoost")
)


In [30]:
results.to_csv(Path().resolve().joinpath("results/XGBoost_eval.csv"))
results_resample.to_csv(Path().resolve().joinpath("results/XGBoost_eval_resampled.csv"))
